# pytorch的自动求导机制

In [1]:
import torch

In [2]:
x = torch.tensor([2.0], requires_grad = True)
a = torch.tensor([4.0], requires_grad = True)
print(x.grad)
print(a.grad)

None
None


In [3]:
y = x * a

In [4]:
y.backward()  # 反向传播，计算梯度

In [5]:
print(x.grad)
print(a.grad)

tensor([4.])
tensor([2.])


# 简单的线性函数的拟合

In [6]:
# 构造数据
x = torch.randn(100)
y = 3 *x + 2

In [7]:
# 初始化参数
k = torch.tensor([0.0], requires_grad=True)
b = torch.tensor([0.0], requires_grad=True)

In [8]:
# 迭代一次
y_pre = k*x + b
y_pre

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.], grad_fn=<AddBackward0>)

In [9]:
lr = 0.015  # 设置学习速率
# 定义损失函数为MSE
def loss_fun(y_true, y_pre):
    return torch.sum(torch.square(y_pre-y_true))/len(y_pre)

In [10]:
loss = loss_fun(y, y_pre)

In [11]:
loss

tensor(9.7098, grad_fn=<DivBackward0>)

In [12]:
# 反向传播计算梯度
loss.backward()

In [15]:
# 获取梯度
print(k.grad)
print(b.grad)

tensor([-4.8855])
tensor([-2.3815])


In [16]:
# 更新参数
k.data -= k.grad * lr
b.data -= b.grad * lr

In [17]:
print(k)
print(b)

tensor([0.0733], requires_grad=True)
tensor([0.0357], requires_grad=True)


In [18]:
# 完成一个batch将梯度置零
k.grad.zero_()
b.grad.data.zero_()

tensor([0.])

In [19]:
# 迭代多次
for i in range(1000):
    y_pre = k*x + b
    # print(y_pre)
    loss = loss_fun(y, y_pre)
    loss.backward()
    # 获取梯度
    # print(k.grad)
    # print(b.grad)
    # 更新参数
    k.data -= k.grad * lr
    b.data -= b.grad * lr
    # 每训练10次打印一次记录
    if i % 100 == 0:
        print("Iter: %d, k: %.4f, b: %.4f, training loss: %.4f" %
              (i, k.item(), b.item(), loss.item()))
    k.grad.data.zero_()
    b.grad.data.zero_()

Iter: 0, k: 0.1447, b: 0.0710, training loss: 9.2719
Iter: 100, k: 2.7255, b: 1.7471, training loss: 0.1060
Iter: 200, k: 2.9706, b: 1.9713, training loss: 0.0013
Iter: 300, k: 2.9968, b: 1.9968, training loss: 0.0000
Iter: 400, k: 2.9996, b: 1.9996, training loss: 0.0000
Iter: 500, k: 3.0000, b: 2.0000, training loss: 0.0000
Iter: 600, k: 3.0000, b: 2.0000, training loss: 0.0000
Iter: 700, k: 3.0000, b: 2.0000, training loss: 0.0000
Iter: 800, k: 3.0000, b: 2.0000, training loss: 0.0000
Iter: 900, k: 3.0000, b: 2.0000, training loss: 0.0000


# Pytorch套路

## 定义模型构造类

In [20]:
class SimpleLinear:
    # 初始化参数
    def __init__(self):
        self.k = torch.tensor([0.0], requires_grad=True)
        self.b = torch.tensor([0.0], requires_grad=True)
       
    # 模型前向计算
    def forward(self, x):
        y = self.k * x + self.b
        return y
    
    # 模型参数
    def parameters(self):
        return [self.k, self.b]
    
    # 调用模型自动开始计算
    def __call__(self, x):
        return self.forward(x)

## 定义优化器类

In [21]:
class Optimizer:
    # 初始化参数
    def __init__(self, parameters, lr):
        self.parameters = parameters
        self.lr = lr
        
    # 更新参数
    def step(self):
        for para in self.parameters:
            para.data -= para.grad * self.lr
    
    # 初始化梯度
    def zero_grad(self):
        for para in self.parameters:
            para.grad.zero_()

## 定义损失函数

In [22]:
# 定义损失函数为MSE
def loss_fun(y_true, y_pre):
    return torch.sum(torch.square(y_pre-y_true))/len(y_pre)

## 模型的训练

In [23]:
# 实例化模型
model = SimpleLinear()
# 实例化优化器
opt = Optimizer(model.parameters(), lr=0.015)
for epoch in range(500):
    # 计算模型输出
    output = model(x)
    # 计算损失值
    loss = loss_fun(y, output)
    # 反向传播，根据计算图计算每个参数的梯度
    loss.backward()
    # 参数更新
    opt.step()
    # 参数梯度归零
    opt.zero_grad()
    if epoch % 50 == 0:
        print('Epoch:{0}, k:{1:.4f}, b:{2:.4f}, loss:{3:.6f}'.format(epoch, model.parameters()[0].item(), model.parameters()[1].item(), loss))



Epoch:0, k:0.0733, b:0.0357, loss:9.709770
Epoch:50, k:2.1161, b:1.2598, loss:1.015905
Epoch:100, k:2.7192, b:1.7416, loss:0.110783
Epoch:150, k:2.9086, b:1.9124, loss:0.012179
Epoch:200, k:2.9699, b:1.9707, loss:0.001341
Epoch:250, k:2.9901, b:1.9902, loss:0.000148
Epoch:300, k:2.9967, b:1.9968, loss:0.000016
Epoch:350, k:2.9989, b:1.9989, loss:0.000002
Epoch:400, k:2.9996, b:1.9996, loss:0.000000
Epoch:450, k:2.9999, b:1.9999, loss:0.000000
